# 结构化输出解析器

尽管 Pydantic/JSON 解析器更加强大，但我们最初尝试的数据结构仅包含文本字段。# Structured Output Parser

In [23]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

在这里定义了我们想要接收的响应模式。Here we define the response schema we want to receive.

In [24]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="source", description="source used to answer the user's question, should be a website.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

现在我们得到一个包含响应格式指令的字符串，然后将其插入到我们的提示符中。We now get a string that contains instructions for how the response should be formatted, and we then insert that into our prompt.

In [25]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

现在我们可以使用这个来格式化一个提示符，然后解析返回的结果。We can now use this to format a prompt to send to the language model, and then parse the returned result.

In [26]:
model = OpenAI(temperature=0)

In [27]:
_input = prompt.format_prompt(question="what's the capital of france?")
output = model(_input.to_string())

In [28]:
output_parser.parse(output)

{'answer': 'Paris',
 'source': 'https://www.worldatlas.com/articles/what-is-the-capital-of-france.html'}

这里是在聊天模型中使用它的一个示例。And here's an example of using this in a chat model

In [29]:
chat_model = ChatOpenAI(temperature=0)

In [30]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("answer the users question as best as possible.\n{format_instructions}\n{question}")  
    ],
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [31]:
_input = prompt.format_prompt(question="what's the capital of france?")
output = chat_model(_input.to_messages())

In [32]:
output_parser.parse(output.content)

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/Paris'}